In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [8]:
train = pd.read_csv("input/training.csv")
test  = pd.read_csv("input/test.csv")

In [9]:
features = list(train.columns[1:-5])

In [10]:
rf = RandomForestClassifier(n_estimators=300, random_state=1)
rf.fit(train[features], train["signal"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [11]:
params = {"objective": "binary:logistic",
          "eta": 0.4,
          "max_depth": 6,
          "min_child_weight": 2,
          "silent": 1,
          "subsample": 0.6,
          "colsample_bytree": 0.6,
          "seed": 1}
num_trees=250
gbm = xgb.train(params, xgb.DMatrix(train[features], train["signal"]), num_trees)

In [12]:
test_probs = (rf.predict_proba(test[features])[:,1] +
              gbm.predict(xgb.DMatrix(test[features])))/2
submission = pd.DataFrame({"id": test["id"], "prediction": test_probs})
submission.to_csv("output/ben_hamner_rf_xgboost_submission.csv", index=False)